In [11]:
## Set-up.
### Libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [12]:
DATA_PATH = '../dataset/'


# Feature engineering

In [13]:
data = pd.read_csv(os.path.join(DATA_PATH, 'Train.csv'))
data.dropna(axis=0, subset=['income_above_limit'], inplace=True)

In [14]:

def edu_tr(x):  #!!!!!
    c1 = []
    if x.strip().lower() in ['less than 1st grade', '1st 2nd 3rd or 4th grade', '5th or 6th grade', '7th and 8th grade',
                             '9th grade', '10th grade', '11th grade', '12th grade no diploma']: x = 'no_highschool'
    if x.strip().lower() in ['high school graduate']: x = 'highschool'
    if x.strip().lower() in ['some college but no degree']: x = 'somecollege'
    if x.strip().lower() in ['associates degree-academic program']: x = 'assc_acad'
    if x.strip().lower() in ['associates degree-occup /vocational']: x = 'assc_voc'
    if x.strip().lower() in ['bachelors degree(ba ab bs)']: x = 'bachelors'
    if x.strip().lower() in ['masters degree(ma ms meng med msw mba)']: x = 'masters'
    if x.strip().lower() in ['prof school degree (md dds dvm llb jd)', 'doctorate degree(phd edd)']: x = 'doctorate'
    return x


def hisp_tr(x):
    if x.strip().lower() in ['all other']:
        x = 'No'
    else:
        x = 'Yes'
    return x


def unempl_tr(x):
    if x.strip().lower() in ['not in labor force', 'unemployed full-time']: x = 'Unemployed'
    if x.strip().lower() in ['pt for non-econ reasons usually ft', 'pt for econ reasons usually pt',
                             'unemployed part- time', 'pt for econ reasons usually ft']: x = 'Part-time employed'
    return x


def feature_engineering(data):
    data_pr = data.copy()
    drop_col = ['ID', 'education_institute', 'unemployment_reason', 'is_labor_union', 'industry_code',
                'occupation_code', 'total_employed', 'household_stat', 'under_18_family',
                'veterans_admin_questionnaire', 'gains', 'losses', 'stocks_status', 'mig_year', 'country_of_birth_own',
                'country_of_birth_father', 'country_of_birth_mother', 'migration_code_change_in_msa',
                'migration_prev_sunbelt', 'migration_code_move_within_reg', 'migration_code_change_in_reg',
                'residence_1_year_ago', 'old_residence_reg', 'old_residence_state', 'importance_of_record']
    data_pr = data_pr.loc[data.age >= 18]
    data_pr.age = pd.cut(data.age, bins=[17, 24, 32, 45, 62, np.inf], right=True,
                         labels=['18_24', '25_32', '33_45', '46_62', '63+'])
    data_pr.education = data_pr.education.map(lambda x: edu_tr(x))
    data_pr['class'] = data_pr['class'].fillna('NA')
    data_pr['is_hispanic'] = data_pr['is_hispanic'].map(lambda x: hisp_tr(x))
    data_pr['employment_commitment'] = data_pr['employment_commitment'].map(lambda x: unempl_tr(x))
    data_pr['wage_per_hour'] = pd.cut(data_pr['wage_per_hour'], bins=[-1, 10, 50, 100, 500, 1000, np.inf], right=True,
                                      labels=['0_10', '11_50', '51-100', '101_500', '501_1000', '1000+'])
    data_pr['working_week_per_year'] = pd.cut(data_pr['working_week_per_year'], bins=[-1, 0, 25, 47, np.inf],
                                              right=True,
                                              labels=['weeks_worked_0', 'weeks_worked_1_25', 'weeks_worked_26_47',
                                                      'weeks_worked_48_52'])
    data_pr['occupation_code_main'] = data_pr['occupation_code_main'].fillna('NA')
    data_pr['employment_stat'] = data_pr['employment_stat'].astype("category")
    data_pr['vet_benefit'] = data_pr['vet_benefit'].astype("category")
    data_pr['investments'] = data[['gains', 'losses', 'stocks_status']].abs().any(axis=1)

    data_pr = data_pr.drop(drop_col, axis=1)
    for i in data_pr.columns:
        if data_pr[i].dtype.name == 'category': data_pr[i] = data_pr[i].astype('str')
    return data_pr


In [15]:
data1 = feature_engineering(data)

In [16]:
data1

,age,gender,education,class,marital_status,race,is_hispanic,employment_commitment,employment_stat,wage_per_hour,working_week_per_year,industry_code_main,occupation_code_main,household_summary,vet_benefit,tax_status,citizenship,income_above_limit,investments
0,63+,Female,highschool,NA,Widowed,White,No,Unemployed,0,0_10,weeks_worked_48_52,Not in universe or children,NA,Householder,2,Head of household,Native,Below limit,True
1,63+,Female,highschool,NA,Widowed,White,No,Children or Armed Forces,0,0_10,weeks_worked_0,Not in universe or children,NA,Householder,2,Single,Native,Below limit,False
2,18_24,Male,no_highschool,Federal government,Never married,Black,No,Children or Armed Forces,0,101_500,weeks_worked_1_25,Hospital services,Adm support including clerical,Child 18 or older,2,Single,Native,Below limit,False
4,63+,Male,highschool,NA,Married-civilian spouse present,White,No,Unemployed,0,0_10,weeks_worked_0,Not in universe or children,NA,Spouse of householder,2,Joint both 65+,Native,Below limit,False
5,33_45,Male,bachelors,Private,Married-civilian spouse present,Asian or Pacific Islander,No,Children or Armed Forces,0,501_1000,weeks_worked_48_52,Retail trade,Other service,Householder,2,Joint both under 65,Foreign born- Not a citizen of U S,Below limit,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209492,33_45,Female,no_highschool,NA,Married-civilian spouse present,White,Yes,Unemployed,0,0_10,weeks_worked_0,Not in universe or children,NA,Spouse of householder,2,Joint both under 65,Foreign born- Not a citizen of U S,Below limit,False
209493,25_32,Male,assc_voc,State government,Never married,White,No,Full-time schedules,0,0_10,weeks_worked_26_47,Public administration,Other service,Child 18 or older,2,Single,Native,Below limit,True
209495,63+,Male,bachelors,NA,Married-civilian spouse present,White,No,Unemployed,0,0_10,weeks_worked_0,Not in universe or children,NA,Householder,2,Joint both 65+,Native,Below limit,True
209497,25_32,Female,somecollege,Private,Married-civilian spouse present,White,No,Children or Armed Forces,0,501_1000,weeks_worked_48_52,Wholesale trade,Handlers equip cleaners etc,Spouse of householder,2,Joint both under 65,Native,Below limit,False


In [20]:
data1.to_parquet(os.path.join(DATA_PATH,'data_eng.gzip'))

In [18]:
data

,ID,age,gender,education,class,education_institute,marital_status,race,is_hispanic,employment_commitment,...,country_of_birth_mother,migration_code_change_in_msa,migration_prev_sunbelt,migration_code_move_within_reg,migration_code_change_in_reg,residence_1_year_ago,old_residence_reg,old_residence_state,importance_of_record,income_above_limit
0,ID_TZ0000,79,Female,High school graduate,NaN,NaN,Widowed,White,All other,Not in labor force,...,US,?,?,?,?,NaN,NaN,NaN,1779.74,Below limit
1,ID_TZ0001,65,Female,High school graduate,NaN,NaN,Widowed,White,All other,Children or Armed Forces,...,US,unchanged,NaN,unchanged,unchanged,Same,NaN,NaN,2366.75,Below limit
2,ID_TZ0002,21,Male,12th grade no diploma,Federal government,NaN,Never married,Black,All other,Children or Armed Forces,...,US,unchanged,NaN,unchanged,unchanged,Same,NaN,NaN,1693.42,Below limit
3,ID_TZ0003,2,Female,Children,NaN,NaN,Never married,Asian or Pacific Islander,All other,Children or Armed Forces,...,India,unchanged,NaN,unchanged,unchanged,Same,NaN,NaN,1380.27,Below limit
4,ID_TZ0004,70,Male,High school graduate,NaN,NaN,Married-civilian spouse present,White,All other,Not in labor force,...,US,?,?,?,?,NaN,NaN,NaN,1580.79,Below limit
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209494,ID_TZ99995,16,Female,7th and 8th grade,NaN,High school,Never married,White,All other,Not in labor force,...,US,?,?,?,?,NaN,NaN,NaN,2170.60,Below limit
209495,ID_TZ99996,78,Male,Bachelors degree(BA AB BS),NaN,NaN,Married-civilian spouse present,White,All other,Not in labor force,...,Canada,?,?,?,?,NaN,NaN,NaN,1029.57,Below limit
209496,ID_TZ99997,8,Female,Children,NaN,NaN,Never married,White,All other,Children or Armed Forces,...,US,?,?,?,?,NaN,NaN,NaN,791.61,Below limit
209497,ID_TZ99998,26,Female,Some college but no degree,Private,NaN,Married-civilian spouse present,White,All other,Children or Armed Forces,...,US,unchanged,NaN,unchanged,unchanged,Same,NaN,NaN,3530.26,Below limit


In [19]:
data['education'].map(lambda x: edu_tr(x)).unique()

array(['highschool', 'no_highschool', ' Children', 'bachelors', 'masters',
       'assc_acad', 'somecollege', 'assc_voc', 'doctorate'], dtype=object)

In [10]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 150912 entries, 0 to 209498
Data columns (total 19 columns):
 #   Column                 Non-Null Count   Dtype   
---  ------                 --------------   -----   
 0   age                    150912 non-null  category
 1   gender                 150912 non-null  object  
 2   education              150912 non-null  object  
 3   class                  150912 non-null  object  
 4   marital_status         150912 non-null  object  
 5   race                   150912 non-null  object  
 6   is_hispanic            150912 non-null  object  
 7   employment_commitment  150912 non-null  object  
 8   employment_stat        150912 non-null  category
 9   wage_per_hour          150912 non-null  category
 10  working_week_per_year  150912 non-null  category
 11  industry_code_main     150912 non-null  object  
 12  occupation_code_main   150912 non-null  object  
 13  household_summary      150912 non-null  object  
 14  vet_benefit            15